<a href="https://colab.research.google.com/github/navseducation/Gen-AI-Purdue-Course/blob/main/Word2vec_gensim_movie_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load Gensim Library

In [ ]:
!pip install gensim

In [ ]:
import gensim
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

### Load Text Data

Data can be downloaded from https://www.kaggle.com/c/word2vec-nlp-tutorial/data

In [ ]:
#This is needed only if you have uploaded data to Google drive
from google.colab import drive
drive.mount('/gdrive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import pandas as pd

#change file path to point to where you have stored the zip file.
df = pd.read_csv('unlabeledTrainData.tsv.zip', header=0, delimiter="\t", quoting=3)
#df = pd.read_csv('unlabeledTrainData.tsv.zip', header=0, delimiter="\t", quoting=3)
print('Number of examples in Dataset: ', df.shape)
df.head()

Number of examples in Dataset:  (50000, 2)


,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


### Function to Clean up data

In [ ]:
import re, string

def clean_str(string):
  """
  String cleaning before vectorization
  """
  try:
    string = re.sub(r'^https?:\/\/<>.*[\r\n]*', '', string, flags=re.MULTILINE)
    string = re.sub(r"[^A-Za-z]", " ", string)
    words = string.strip().lower().split()
    words = [w for w in words if len(w)>=1]
    return " ".join(words)
  except:
    return ""

### Clean the Data using routine above

In [ ]:
df['clean_review'] = df['review'].apply(clean_str)
df.head()

,id,review,clean_review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was...",watching time chasers it obvious that it was m...
1,"""45057_0""","""I saw this film about 20 years ago and rememb...",i saw this film about years ago and remember i...
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B...",minor spoilers br br in new york joan barnard ...
3,"""7161_0""","""I went to see this film with a great deal of ...",i went to see this film with a great deal of e...
4,"""43971_0""","""Yes, I agree with everyone on this site this ...",yes i agree with everyone on this site this mo...


In [ ]:
df['clean_review'][0].split(' ')

['watching',
 'time',
 'chasers',
 'it',
 'obvious',
 'that',
 'it',
 'was',
 'made',
 'by',
 'a',
 'bunch',
 'of',
 'friends',
 'maybe',
 'they',
 'were',
 'sitting',
 'around',
 'one',
 'day',
 'in',
 'film',
 'school',
 'and',
 'said',
 'hey',
 'let',
 's',
 'pool',
 'our',
 'money',
 'together',
 'and',
 'make',
 'a',
 'really',
 'bad',
 'movie',
 'or',
 'something',
 'like',
 'that',
 'what',
 'ever',
 'they',
 'said',
 'they',
 'still',
 'ended',
 'up',
 'making',
 'a',
 'really',
 'bad',
 'movie',
 'dull',
 'story',
 'bad',
 'script',
 'lame',
 'acting',
 'poor',
 'cinematography',
 'bottom',
 'of',
 'the',
 'barrel',
 'stock',
 'music',
 'etc',
 'all',
 'corners',
 'were',
 'cut',
 'except',
 'the',
 'one',
 'that',
 'would',
 'have',
 'prevented',
 'this',
 'film',
 's',
 'release',
 'life',
 's',
 'like',
 'that']

### Convert Review to a Word List

In [ ]:
#List to hold all words in each review
documents = []

#Iterate over each review
for doc in df['clean_review']:
    documents.append(doc.split(' '))

print(len(documents))
print(documents[0])

50000
['watching', 'time', 'chasers', 'it', 'obvious', 'that', 'it', 'was', 'made', 'by', 'a', 'bunch', 'of', 'friends', 'maybe', 'they', 'were', 'sitting', 'around', 'one', 'day', 'in', 'film', 'school', 'and', 'said', 'hey', 'let', 's', 'pool', 'our', 'money', 'together', 'and', 'make', 'a', 'really', 'bad', 'movie', 'or', 'something', 'like', 'that', 'what', 'ever', 'they', 'said', 'they', 'still', 'ended', 'up', 'making', 'a', 'really', 'bad', 'movie', 'dull', 'story', 'bad', 'script', 'lame', 'acting', 'poor', 'cinematography', 'bottom', 'of', 'the', 'barrel', 'stock', 'music', 'etc', 'all', 'corners', 'were', 'cut', 'except', 'the', 'one', 'that', 'would', 'have', 'prevented', 'this', 'film', 's', 'release', 'life', 's', 'like', 'that']


In [ ]:
print(len(documents[0]))

90


In [ ]:
len(documents[1])

82

### Build the Model

In [ ]:
#Build the model
model = gensim.models.Word2Vec(documents, #Word list
                               min_count=5, #Ignore all words with total frequency lower than this
                               workers=6, #Number of CPU Cores
                               vector_size=30,  #Embedding size
                               window=5, #Maximum Distance between current and predicted word
                               epochs=10  #Number of iterations over the text corpus
                              )

# Exploring the model

### How many words in the model

### Get an embedding for a word

In [ ]:
model.wv['great']

array([-1.9424357e+00,  2.1404312e+00, -2.5971034e+00, -2.2209520e+00,
       -8.1891948e-01, -4.9431759e-01, -1.9749097e+00,  2.5203962e+00,
        3.7257774e+00,  1.6287118e-01, -7.7316338e-01,  1.0920780e-01,
        5.2986735e-01,  3.5708600e-01,  3.1625209e+00,  4.4290609e+00,
        2.7523293e+00,  4.1587214e+00, -2.4624209e+00,  4.6654501e+00,
        1.7440808e+00, -2.7851543e+00,  1.7101963e+00,  3.0741794e+00,
        2.0851676e+00, -4.2166452e+00,  5.7515204e-01, -9.1815478e-04,
       -5.0058895e-01,  3.8452015e+00], dtype=float32)

In [ ]:
model.wv['amazing']

array([-0.67832536,  1.5699481 , -0.63711715, -1.8408027 , -2.55018   ,
        0.2478128 , -0.69779724,  1.6226641 ,  3.3129272 ,  0.44982693,
       -2.0713723 , -1.2038429 , -0.259252  , -1.3458554 ,  1.6881342 ,
        3.1647358 ,  2.6609871 ,  3.377169  , -0.14035086,  3.8639238 ,
        3.2351465 , -3.9280946 , -2.3333888 ,  2.8832068 ,  1.0821061 ,
       -3.643941  ,  1.269224  ,  1.1912572 ,  0.10719376,  0.7017195 ],
      dtype=float32)

### Finding Words which have similar meaning

In [ ]:
model.wv.most_similar('amazing')

[('incredible', 0.9010778069496155),
 ('awesome', 0.8972740769386292),
 ('fantastic', 0.8483001589775085),
 ('brilliant', 0.8324704170227051),
 ('exceptional', 0.8279510140419006),
 ('outstanding', 0.8272615075111389),
 ('great', 0.8178824782371521),
 ('wonderful', 0.8119750022888184),
 ('excellent', 0.8055941462516785),
 ('perfect', 0.7951630353927612)]

In [ ]:
model.wv.most_similar('delhi')

[('cincinnati', 0.883975088596344),
 ('copenhagen', 0.877750813961029),
 ('travelled', 0.8701186776161194),
 ('elope', 0.8570157885551453),
 ('ventured', 0.83570396900177),
 ('donegal', 0.8345761299133301),
 ('embarked', 0.8334171175956726),
 ('pasture', 0.8321858048439026),
 ('winery', 0.8286857008934021),
 ('portugal', 0.8286079168319702)]

### Find the word which is not like others

### Saving the model

In [ ]:
model.save('word2vec-movie-50')

In [ ]:
#Load model from memory
model = gensim.models.Word2Vec.load('word2vec-movie-50')

1. Equation king + man = queen + ?
2. In this case there may not be enough data for this equation

In [ ]:
model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=10)

[('saint', 0.7291987538337708),
 ('el', 0.7206029891967773),
 ('milady', 0.6983327865600586),
 ('facto', 0.6894933581352234),
 ('reagan', 0.683952271938324),
 ('xi', 0.6828990578651428),
 ('rio', 0.6754740476608276),
 ('austria', 0.6720345616340637),
 ('colman', 0.6701590418815613),
 ('cuba', 0.6698727011680603)]

In [ ]:
model.wv.most_similar(positive=['woman', 'hero'], negative=['man'], topn=5)

[('heroine', 0.8993502855300903),
 ('protagonist', 0.8276101350784302),
 ('antagonist', 0.7001104950904846),
 ('argument', 0.6695447564125061),
 ('predicament', 0.6595082879066467)]

In [ ]:
model.wv.most_similar(positive=['woman','father'], negative=['man'])

[('mother', 0.9199700951576233),
 ('husband', 0.8869858980178833),
 ('daughter', 0.886826753616333),
 ('wife', 0.8730366826057434),
 ('grandmother', 0.861428439617157),
 ('sister', 0.8505318760871887),
 ('aunt', 0.8252345323562622),
 ('child', 0.8144773244857788),
 ('marriage', 0.8135500550270081),
 ('parents', 0.8077060580253601)]

In [ ]:
model.wv['king'] + model.wv['man'] - model.wv['queen']

array([ 4.882239  ,  1.5166391 ,  3.2138824 , -0.3846364 ,  2.7624085 ,
        1.4639171 , -1.3386823 ,  1.8275785 , -2.3054118 ,  2.9164822 ,
       -1.3305513 , -1.6388509 ,  3.9274638 ,  0.32863474, -1.1456629 ,
       -5.609324  ,  0.22982876, -0.91295826, -4.6305885 , -2.8013325 ,
       -2.5239637 , -4.4960194 ,  1.1209521 ,  3.1037326 ,  4.313352  ,
        0.18896341,  0.60024667, -1.267221  ,  2.3559675 , -1.8934937 ],
      dtype=float32)